<a href="https://colab.research.google.com/github/seunghee0518/AI_python/blob/main/EfficientNet_last.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## efficientNet_v2
* timm > efficientnetv2_rw_s
* 드롭아웃 0.3
* 옵티마이저 sam
* 5fold
* 이미지셋 : 기본 + 변형 + 오류데이터*3
* 이미지 크기 : 300 * 300
* 정규화 : 이미지넷 평균/표준편차 > mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]

> albumentations
* albumentations.RandomCrop(224, 224),
* albumentations.HorizontalFlip(p=0.5),
* albumentations.Rotate(limit=90, p=.5)
* albumentations.VerticalFlip(p=0.5)

In [1]:
## 코랩 - 구글 드라이브
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# ## 이미지 셋 정규화 수치 확인
# # !pip install -q -U albumentations==0.4.6
# # !echo "$(pip freeze | grep albumentations) is successfully installed"

# import warnings
# warnings.filterwarnings('ignore')

# from glob import glob
# import pandas as pd
# import numpy as np 
# from tqdm import tqdm
# import cv2

# import os
# import timm
# import random

# import torch
# from torch.utils.data import Dataset, DataLoader
# import torch.nn as nn
# import torchvision.transforms as transforms
# from sklearn.metrics import f1_score, accuracy_score
# import time

# from sklearn.model_selection import StratifiedKFold

# import albumentations
# from albumentations.pytorch.transforms import ToTensorV2
# from PIL import Image


# def img_load(path):
#     img = cv2.imread(path)[:,:,::-1]
#     img = cv2.resize(img, (300, 300))#,interpolation = cv2.INTER_AREA)
#     return img

# transform = albumentations.Compose([ShiftScaleRotate(rotate_limit=20, p=0.5, border_mode=cv2.BORDER_CONSTANT),])
#     # albumentations.RandomCrop(300, 300),
#     #                                albumentations.HorizontalFlip(p=0.5),
#     #                                albumentations.Rotate(limit=90, p=.5),
#     #                                albumentations.VerticalFlip(p=0.5),])

# train_png = sorted(glob(f'{path}/train/*.png'))
# train_imgs = [img_load(m) for m in tqdm(train_png)]
# len(train_imgs)

# train_df = pd.read_csv(f"{path}/train_df.csv")
# train_labels = train_df["label"] # 레이블순서는 이미지 파일 순서대로임

# ## 레이블별 키값
# label_unique = sorted(np.unique(train_labels))
# label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))} # 오름차순으로 레이블별로 숫자 부여(0부터 시작)

# train_labels = [label_unique[k] for k in tqdm(train_labels)]

# ## not_good ## 
# # not_good_imgs = [f'{path}/train/{x}' for x in train_df[train_df.state != 'good'].file_name.tolist()]
# # not_good_imgs = [img_load(x) for x in tqdm(not_good_imgs)]
# # not_good_label = train_df[train_df.state != 'good'].label.tolist()
# # not_good_labels = [label_unique[k] for k in not_good_label]

# train_imgs2 = (train_imgs * 1 )#+ (not_good_imgs*3)
# train_imgs2 = [transform(image=x)['image'] for x in tqdm(train_imgs2)]

# all_train_imgs = train_imgs + train_imgs2

# meanRGB = [np.mean(x, axis=(0,1)) for x in tqdm(all_train_imgs)]
# stdRGB = [np.std(x, axis=(0,1)) for x in (all_train_imgs)]

# meanR = np.mean([m[0] for m in tqdm(meanRGB)])/255
# meanG = np.mean([m[1] for m in tqdm(meanRGB)])/255
# meanB = np.mean([m[2] for m in tqdm(meanRGB)])/255

# stdR = np.mean([s[0] for s in tqdm(stdRGB)])/255
# stdG = np.mean([s[1] for s in tqdm(stdRGB)])/255
# stdB = np.mean([s[2] for s in tqdm(stdRGB)])/255

# # print('train-imageset  평균   R : {:.3f} G : {:.3f} B : {:.3f}'.format(meanR,meanG,meanB))
# # print('train-imageset 표준편차 R : {:.3f} G : {:.3f} B : {:.3f}'.format(stdR,stdG,stdB))

# '''
# train-imageset 평균 0.43452070328285136 0.40755165070821625 0.39973605673083556
# train-imageset 표준편차 0.18461959399416333 0.17982346710429026 0.16928450977358847
# '''

In [3]:
## 학습 코드
!pip install timm
import timm
!pip install -q -U albumentations==0.4.6
!echo "$(pip freeze | grep albumentations) is successfully installed"

from albumentations.pytorch.transforms import ToTensorV2

# from pprint import pprint
# model_names = timm.list_models(pretrained=True)
# pprint(model_names)

import warnings
warnings.filterwarnings('ignore')

import torch
from glob import glob
from tqdm import tqdm
import cv2

#import timm
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score, accuracy_score
import gc
import pandas as pd
import numpy as np
import json
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
import time
import os
import copy
import random
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
from PIL import Image
from sklearn.model_selection import StratifiedKFold

#!pip install albumentations==0.4.6
import albumentations
#from albumentations.pytorch import ToTensorV2
# from efficientnet_pytorch import EfficientNet
# model_name = 'efficientnet-b0'  # b5

# image_size = EfficientNet.get_image_size(model_name)
# print(image_size)
# model = EfficientNet.from_pretrained(model_name, num_classes=88)

albumentations==0.4.6 is successfully installed


In [4]:
## 기본 설정
device = torch.device('cuda')
batch_size  = 32
random_seed = 1234
img_size = 300
torch.manual_seed(random_seed)

def main(seed = random_seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True
    
main(random_seed)

## transforms 함수 정리
* transforms.ToPILImage() - csv 파일로 데이터셋을 받을 경우, PIL image로 바꿔준다.
* transforms.CenterCrop(size) - 가운데 부분을 size 크기로 자른다.
* transforms.Grayscale(num_output_channels=1) - grayscale로 변환한다.
* transforms.RandomAffine(degrees) - 랜덤으로 affine 변형을 한다.
* transforms.RandomCrop(size) -이미지를 랜덤으로 아무데나 잘라 size 크기로 출력한다.
* transforms.RandomResizedCrop(size) - 이미지 사이즈를 size로 변경한다
* transforms.Resize(size) - 이미지 사이즈를 size로 변경한다
* transforms.RandomRotation(degrees) 이미지를 랜덤으로 degrees 각도로 회전한다.
* transforms.RandomResizedCrop(size, scale=(0.08, 1.0), ratio=(0.75, 1.3333333333333333)) - 이미지를 랜덤으로 변형한다.
* transforms.RandomVerticalFlip(p=0.5) - 이미지를 랜덤으로 수직으로 뒤집는다. p =0이면 뒤집지 않는다.
* transforms.RandomHorizontalFlip(p=0.5) - 이미지를 랜덤으로 수평으로 뒤집는다.
* transforms.ToTensor() - 이미지 데이터를 tensor로 바꿔준다.
* transforms.Normalize(mean, std, inplace=False) - 이미지를 정규화한다.

In [5]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1] # Return type:	numpy.ndarray / 기본적으로 BGR로 불러옴, RGB 변환함
    img = cv2.resize(img, (300, 300))
    return img

class Custom_dataset_Albumentations(Dataset):
    """__init__ and __len__ functions are the same as in TorchvisionDataset"""
    def __init__(self, img_list, labels, transform=None, mode='train'):
        self.img_list = img_list
        self.labels = labels
        self.transform = transform
        self.mode=mode
        
    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        label = self.labels[idx]
        img   = self.img_list[idx]

        if self.mode=='train':
          if idx < 4277: # 변경 없는 이미지
            img = transforms.ToTensor()(img) # PIL or ndarray -> tensor
            img = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(img)

          else : # 랜덤 변경된 이미지 추가됨 
            if self.transform:
                #img = Image.fromarray(img)
                augmented = self.transform(image=img)
                img = augmented['image']

        if self.mode=='test':
          img = transforms.ToTensor()(img)
          img = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(img)
        return img, label

albumentations_transform = albumentations.Compose([albumentations.RandomCrop(300, 300),
                                                   albumentations.HorizontalFlip(p=0.5),
                                                   albumentations.Rotate(limit=90, p=.5),#defalt : interpolation=1, border_mode=4, always_apply=False, p=0.5)
                                                   albumentations.VerticalFlip(p=0.5),
                                                  #  albumentations.OneOf([albumentations.HorizontalFlip(p=1),
                                                  #                        albumentations.RandomRotate90(p=1),
                                                  #                        albumentations.VerticalFlip(p=1),
                                                  #                        # albumentations.MotionBlur(p=1), # 왜곡 
                                                  #                        # albumentations.OpticalDistortion(p=1), 
                                                  #                        # albumentations.GaussNoise(p=1),         
                                                  #                         ], p=1),
                                                   albumentations.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225],),
                                                   albumentations.pytorch.ToTensorV2()])

# 형광님 vit 버전
# albumentations_transform = albumentations.Compose([albumentations.RandomCrop(224, 224),
#                                                    albumentations.HorizontalFlip(),
#                                                    albumentations.VerticalFlip(),
#                                                    albumentations.Rotate(45),
#                                                   #  albumentations.MotionBlur(p=1), # 왜곡
#                                                   #  albumentations.OpticalDistortion(p=1),
#                                                   #  albumentations.GaussNoise(p=1) 
#                                                    albumentations.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225],),
#                                                    transforms.ToTensorV2()])

# 버전3 (버전2 : mean=[0.432, 0.402, 0.393], std=[0.182, 0.174 , 0.163] )
# albumentations_transform = albumentations.Compose([albumentations.HorizontalFlip(p=0.5),
#                                                    albumentations.VerticalFlip(p=0.5),
#                                                    albumentations.ShiftScaleRotate(shift_limit=(-0.1, 0.1), scale_limit=(0.0, 0.3), rotate_limit=[-45, 45], p=0.5, border_mode=cv2.INTER_NEAREST),
#                                                    albumentations.Normalize(mean=[0.432, 0.402, 0.393], std=[0.182, 0.174 , 0.163],),
#                                                    albumentations.pytorch.ToTensorV2()])

##### 옵티마이저 ####
class SAM(torch.optim.Optimizer):
    def __init__(self, params, base_optimizer, rho=0.05, **kwargs):
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"

        defaults = dict(rho=rho, **kwargs)
        super(SAM, self).__init__(params, defaults)

        self.base_optimizer = base_optimizer(self.param_groups, **kwargs)
        self.param_groups = self.base_optimizer.param_groups

    @torch.no_grad()
    def first_step(self, zero_grad=False):
        grad_norm = self._grad_norm()
        for group in self.param_groups:
            scale = group["rho"] / (grad_norm + 1e-12)

            for p in group["params"]:
                if p.grad is None: continue
                e_w = p.grad * scale.to(p)
                p.add_(e_w)  # climb to the local maximum "w + e(w)"
                self.state[p]["e_w"] = e_w

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=False):
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None: continue
                p.sub_(self.state[p]["e_w"])  # get back to "w" from "w + e(w)"

        self.base_optimizer.step()  # do the actual "sharpness-aware" update

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def step(self, closure=None):
        assert closure is not None, "Sharpness Aware Minimization requires closure, but it was not provided"
        closure = torch.enable_grad()(closure)  # the closure should do a full forward-backward pass

        self.first_step(zero_grad=True)
        closure()
        self.second_step()

    def _grad_norm(self):
        shared_device = self.param_groups[0]["params"][0].device  # put everything on the same device, in case of model parallelism
        norm = torch.norm(
                    torch.stack([
                        p.grad.norm(p=2).to(shared_device)
                        for group in self.param_groups for p in group["params"]
                        if p.grad is not None
                    ]),
                    p=2
               )
        return norm
import math
from torch.optim.lr_scheduler import _LRScheduler

class CosineAnnealingWarmUpRestarts(_LRScheduler):
    def __init__(self, optimizer, T_0, T_mult=1, eta_max=0.1, T_up=0, gamma=1., last_epoch=-1):
        if T_0 <= 0 or not isinstance(T_0, int):
            raise ValueError("Expected positive integer T_0, but got {}".format(T_0))
        if T_mult < 1 or not isinstance(T_mult, int):
            raise ValueError("Expected integer T_mult >= 1, but got {}".format(T_mult))
        if T_up < 0 or not isinstance(T_up, int):
            raise ValueError("Expected positive integer T_up, but got {}".format(T_up))
        self.T_0 = T_0
        self.T_mult = T_mult
        self.base_eta_max = eta_max
        self.eta_max = eta_max
        self.T_up = T_up
        self.T_i = T_0
        self.gamma = gamma
        self.cycle = 0
        self.T_cur = last_epoch
        super(CosineAnnealingWarmUpRestarts, self).__init__(optimizer, last_epoch)
    
    def get_lr(self):
        if self.T_cur == -1:
            return self.base_lrs
        elif self.T_cur < self.T_up:
            return [(self.eta_max - base_lr)*self.T_cur / self.T_up + base_lr for base_lr in self.base_lrs]
        else:
            return [base_lr + (self.eta_max - base_lr) * (1 + math.cos(math.pi * (self.T_cur-self.T_up) / (self.T_i - self.T_up))) / 2
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.T_cur = self.T_cur + 1
            if self.T_cur >= self.T_i:
                self.cycle += 1
                self.T_cur = self.T_cur - self.T_i
                self.T_i = (self.T_i - self.T_up) * self.T_mult + self.T_up
        else:
            if epoch >= self.T_0:
                if self.T_mult == 1:
                    self.T_cur = epoch % self.T_0
                    self.cycle = epoch // self.T_0
                else:
                    n = int(math.log((epoch / self.T_0 * (self.T_mult - 1) + 1), self.T_mult))
                    self.cycle = n
                    self.T_cur = epoch - self.T_0 * (self.T_mult ** n - 1) / (self.T_mult - 1)
                    self.T_i = self.T_0 * self.T_mult ** (n)
            else:
                self.T_i = self.T_0
                self.T_cur = epoch
                
        self.eta_max = self.base_eta_max * (self.gamma**self.cycle)
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

class Network(nn.Module):
    def __init__(self,mode = 'train'):
        super(Network, self).__init__()
        self.mode = mode
        if self.mode == 'train':
          self.model = timm.create_model('efficientnetv2_rw_s', pretrained=True, num_classes=88, drop_path_rate = 0.3)
        if self.mode == 'test':
          self.model = timm.create_model('efficientnetv2_rw_s', pretrained=True, num_classes=88, drop_path_rate = 0)
        
    def forward(self, x):
        x = self.model(x)
        return x

### f1 스코어 함수 
def score_function(real, pred): # 라이브러리 > sklearn.metrics 
    score = f1_score(real, pred, average="macro")
    return score

## 이미지 로드 및 전처리

In [5]:
### 이미지 로드 및 전처리
train_imgs = sorted(glob('/content/drive/Othercomputers/내 MacBook Pro/open/train/*.png'))
train_imgs = [img_load(x) for x in tqdm(train_imgs)]

#train_imgs = np.load('/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/train_imgs_224.npy')
train_y = pd.read_csv("/content/drive/Othercomputers/내 MacBook Pro/open/train_df.csv")

train_labels = train_y["label"] # 레이블순서는 이미지 파일 순서대로임

## 레이블별 키값
label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))} # 오름차순으로 레이블별로 숫자 부여(0부터 시작)

train_labels = [label_unique[k] for k in tqdm(train_labels)]

## not_good ## 
not_good_imgs = [f'/content/drive/Othercomputers/내 MacBook Pro/open/train/{x}' for x in train_y[train_y.state != 'good'].file_name.tolist()]
not_good_imgs = [img_load(x) for x in tqdm(not_good_imgs)]
not_good_label = train_y[train_y.state != 'good'].label.tolist()
not_good_labels = [label_unique[k] for k in not_good_label]

### 총 이미지 리스트 생성 ### 기본 + 증량 + 오류*10 = 10498
train_imgs = (train_imgs * 2 ) + (not_good_imgs*3)
train_labels = (train_labels * 2 ) + (not_good_labels*3)
print('\n')
print(len(train_imgs), len(train_labels))

100%|██████████| 648/648 [00:18<00:00, 34.11it/s]



10498 10498


## 모델 학습

In [ ]:
# from pprint import pprint
# model_names = timm.list_models(pretrained=True)
# pprint(model_names)

In [6]:
model_version = str(input()) # efficientNet_v2_last

# '''
# 버전5  > efficientNet_b4_ver5
# * 폴드(2(b4)*2(v2))
# * 에포크30 
# * (기본+변형+오류*3)
# * 커스텀정규화(기본+변형+오류*3) :mean=[0.433, 0.405, 0.396], std=[0.183, 0.177, 0.166]
# * 해상도 300

# 버전6 > efficientNet_b4_ver6
# * 폴드(2(b4)*2(v2))
# * 에포크30 
# * (기본+변형)
# * 커스텀정규화(기본+변형) : mean=[0.432, 0.402, 0.393], std=[0.182, 0.174 , 0.163]
# * 해상도 300
# '''

efficientNet_v2_last


In [11]:
### 모델 학습 함수 ####
# 참고 블로그 : https://keep-steady.tistory.com/35
import gc
def re_train_model(model, criterion, optimizer, scheduler, num_epochs, K_idx): # 중간에 끊겼을때 실행하는 함수
    ## 중간 저장된 모델 불러오기
    PATH = f'/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/{model_version}_model_{K_idx}fold.pth'
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    re_epoch = checkpoint['epoch']
    best_model_wts = checkpoint['best_model_wts']
    best_idx = checkpoint['best_idx']
    best_acc = checkpoint['best_acc']
    best_loss = checkpoint['best_loss']
    train_loss = checkpoint['train_loss']
    train_acc = checkpoint['train_acc']
    valid_loss = checkpoint['valid_loss']
    valid_acc = checkpoint['valid_acc']
    total_f1 = checkpoint['total_f1']

    ##한 폴드 별 모델 - 가장 좋은 모델 저장
    print(f'### restart KFold {K_idx} / {n_splits-1} ###')
    one_model_since = time.time() # 하나 모델(폴드)당 시간

    for epoch in range(re_epoch, num_epochs):
        #one_epoch_since = time.time() ### 한 epoch당 시간
        print('Epoch {}/{}'.format(epoch, num_epochs- 1))
        print('-' * 10)
        train_pred=[] # 이거 추가 : f1 스코어 계산
        train_y=[] # 이거 추가

        # Each epoch has a training and validation phase
        for phase in ['train', 'vaild']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss, running_corrects, num_cnt = 0.0, 0, 0
            
            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device) # device = torch.device('cuda')
                labels = labels.to(device) # device = torch.device('cuda')

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase + sam
                    if phase == 'train':
                        loss.backward()
                        optimizer.first_step(zero_grad=True)
                        criterion(model(inputs), labels).backward()
                        optimizer.second_step(zero_grad=True)

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                num_cnt += len(labels)

                train_pred += preds.detach().cpu().numpy().tolist() #이거 추가
                train_y += labels.detach().cpu().data.numpy().tolist() #이거 추가

            if phase == 'train':
                scheduler.step()
            
            epoch_loss = float(running_loss / num_cnt)
            epoch_acc  = float((running_corrects.double() / num_cnt).cpu()*100)
            f1 = score_function(train_y,  train_pred) # score_function or f1_score

            if phase == 'train':
                train_loss.append(epoch_loss)
                train_acc.append(epoch_acc)
            else:
                valid_loss.append(epoch_loss)
                valid_acc.append(epoch_acc)
                total_f1.append(f1)

            print('{} Loss: {:.5f} Acc: {:.5f} macro-f1: {:.5f}'.format(phase, epoch_loss, epoch_acc, f1))
          
            # deep copy the model(최적모델 저장)
            if phase == 'vaild' and epoch_acc > best_acc:
                best_idx = epoch
                best_acc = epoch_acc
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        # 20에포크 단위별 중간 저장 : 체크포인트
        if (epoch+1)%20 == 0 :
            torch.save({'epoch':epoch,
                        'state_dict':model.state_dict(),
                        'optimizer': optimizer.state_dict(),
                        'best_model_wts':best_model_wts,
                        'best_idx':best_idx,
                        'best_acc':best_acc,
                        'best_loss':best_loss,
                        'train_loss' : train_loss,
                        'train_acc' : train_acc,
                        'valid_loss' : valid_loss,
                        'valid_acc' : valid_acc,
                        'total_f1' : total_f1,},
                        #'scaler': scaler.state_dict(),
                         f'/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/{model_version}_model_{K_idx}fold.pth') 

            print('==> best model saved - %d / %.1f'%(best_idx, best_acc))

        # 한 에포크마다 실행 시간 출력하기(누적 시간으로 출력됨)
        time_elapsed = time.time() - one_model_since # 한 에포트당 시간 - 누적
        print('Accumulate epochs training time : {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        print('\n')

        # 한 에포크마다 필요없는 메모리 지우기 : 지우기 효과는 아직 확인 못해봄
        try:      
          gc.collect() # cpu 비움
          torch.cuda.empty_cache() # gpu 비움
        except:
          pass
          
    ## 학습 마무리 후
    fold_time_elapsed = time.time() - one_model_since # 한 모델(폴드)당 시간
    print('one fold - Training complete in {:.0f}m {:.0f}s'.format(fold_time_elapsed // 60, fold_time_elapsed % 60))

    print(f'{K_idx} Best valid loss & Acc: %d - %.1f' %(best_idx, best_acc))
    # load best model weights
    '''
    한 에포크 마다 저장된 최적의 모델 가중치로 조정 후 다음 에포크가 돌아감
    '''
    model.load_state_dict(best_model_wts)
    torch.save({'state_dict':model.state_dict,
                'train_loss' : train_loss,
                'train_acc' : train_acc,
                'valid_loss' : valid_loss,
                'valid_acc' : valid_acc,
                'total_f1' : total_f1,},
                f'/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/{model_version}_final_best_model_{K_idx}fold.pth') 
    print(f'{K_idx}-Fold model saved')
    return model, best_idx, best_loss, best_acc, train_loss, train_acc, valid_loss, valid_acc, total_f1


def train_model(model, criterion, optimizer, scheduler, num_epochs, K_idx):
    ##한 폴드 별 모델 - 가장 좋은 모델 저장
    print(f'### KFold {K_idx} / {n_splits-1} ###')
    one_model_since = time.time() # 하나 모델(폴드)당 시간
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    train_loss, train_acc, valid_loss, valid_acc, total_f1 = [], [], [], [], []
    
    for epoch in range(num_epochs):
        #one_epoch_since = time.time() ### 한 epoch당 시간
        print('Epoch {}/{}'.format(epoch, num_epochs- 1))
        print('-' * 10)
        train_pred=[] # 이거 추가 : f1 스코어 계산
        train_y=[] # 이거 추가

        # Each epoch has a training and validation phase
        for phase in ['train', 'vaild']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss, running_corrects, num_cnt = 0.0, 0, 0
            
            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device) # device = torch.device('cuda')
                labels = labels.to(device) # device = torch.device('cuda')

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # # backward + optimize only if in training phase
                    # if phase == 'train':
                    #     loss.backward()
                    #     optimizer.step()

                    # backward + optimize only if in training phase + sam
                    if phase == 'train':
                        loss.backward()
                        optimizer.first_step(zero_grad=True)
                        criterion(model(inputs), labels).backward()
                        optimizer.second_step(zero_grad=True)

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                num_cnt += len(labels)

                train_pred += preds.detach().cpu().numpy().tolist() #이거 추가
                train_y += labels.detach().cpu().data.numpy().tolist() #이거 추가

            if phase == 'train':
                scheduler.step()
            
            epoch_loss = float(running_loss / num_cnt)
            epoch_acc  = float((running_corrects.double() / num_cnt).cpu()*100)
            f1 = score_function(train_y,  train_pred) # score_function or f1_score

            if phase == 'train':
                train_loss.append(epoch_loss)
                train_acc.append(epoch_acc)
            else:
                valid_loss.append(epoch_loss)
                valid_acc.append(epoch_acc)
                total_f1.append(f1)

            print('{} Loss: {:.5f} Acc: {:.5f} macro-f1: {:.5f}'.format(phase, epoch_loss, epoch_acc, f1))
          
            # deep copy the model(최적모델 저장)
            if phase == 'vaild' and epoch_acc > best_acc:
                best_idx = epoch
                best_acc = epoch_acc
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        # 10에포크 단위별 중간 저장 : 체크포인트
        if (epoch+1)%10 == 0 :
            torch.save({'epoch':epoch,
                        'state_dict':model.state_dict(),
                        'optimizer': optimizer.state_dict(),
                        'best_model_wts':best_model_wts,
                        'best_idx':best_idx,
                        'best_acc':best_acc,
                        'best_loss':best_loss,
                        'train_loss' : train_loss,
                        'train_acc' : train_acc,
                        'valid_loss' : valid_loss,
                        'valid_acc' : valid_acc,
                        'total_f1' : total_f1,},
                        #'scaler': scaler.state_dict(),
                         f'/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/{model_version}_model_{K_idx}fold.pth') 

            print('==> best model saved - %d / %.1f'%(best_idx, best_acc))

        # 한 에포크마다 실행 시간 출력하기(누적 시간으로 출력됨)
        time_elapsed = time.time() - one_model_since # 한 에포트당 시간 - 누적
        print('Accumulate epochs training time : {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        print('\n')

        # 한 에포크마다 필요없는 메모리 지우기 : 지우기 효과는 아직 확인 못해봄
        try:      
          gc.collect() # cpu 비움
          torch.cuda.empty_cache() # gpu 비움
        except:
          pass
          
    ## 학습 마무리 후
    fold_time_elapsed = time.time() - one_model_since # 한 모델(폴드)당 시간
    print('one fold - Training complete in {:.0f}m {:.0f}s'.format(fold_time_elapsed // 60, fold_time_elapsed % 60))

    print(f'{K_idx} Best valid loss & Acc: %d - %.1f' %(best_idx, best_acc))
    # load best model weights
    '''
    한 에포크 마다 저장된 최적의 모델 가중치로 조정 후 다음 에포크가 돌아감
    '''
    model.load_state_dict(best_model_wts)
    torch.save({'state_dict':model.state_dict(),
                'train_loss' : train_loss,
                'train_acc' : train_acc,
                'valid_loss' : valid_loss,
                'valid_acc' : valid_acc,
                'total_f1' : total_f1,},
                f'/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/{model_version}_final_best_model_{K_idx}fold.pth') 
    print(f'{K_idx}-Fold model saved')
    return model, best_idx, best_loss, best_acc, train_loss, train_acc, valid_loss, valid_acc, total_f1

# 설정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # set gpu
criterion = nn.CrossEntropyLoss()


# ## 변경된 옵티마이저
# base_optimizer = torch.optim.SGD
# optimizer_ft = SAM(model.parameters(), base_optimizer, lr=1e-6, momentum=0.9) # momentum?
# '''
# lr : optimizer_ft > 최저점 설정
# '''

# lr_scheduler = CosineAnnealingWarmUpRestarts(optimizer_ft, T_0=20, T_mult=1, eta_max=0.1,  T_up=4, gamma=0.5)
# '''
# T_0 : 전체 주기
# T_mult : T_0에 곱할 주기
# eta_max : 시작점
# T_up : 다음 최고점까지 올라갈 주기
# gamma : 다음 주기 때 올라가는 값 계산
# '''


In [ ]:
#######################################################################################
################################## 모델 시작 ############################################
#######################################################################################
num_epochs = 30
n_splits = 5
Skfold = StratifiedKFold(n_splits = n_splits, random_state = random_seed, shuffle=True)
pred_ensemble = []
total_since = time.time() # 전체 실행시간

### 폴드 시작 ###
start_fold =  int(input()) ## 시작할 폴드 설정, 0부터 카운트
restart_state = input('y / n 입력하시오 :') # 학습 중단 될 경우 y
for K_idx, (train_idx, val_idx) in enumerate(Skfold.split(np.array(train_imgs), np.array(train_labels))):
  try:      
    gc.collect() # cpu 비움
    torch.cuda.empty_cache() # gpu 비움
  except:
    pass
  if K_idx >= start_fold: # start_fold = 0일 경우 모든 폴드 실행, start_fold =3 일 경우 3, 4, 5 폴드 실행
    print(f"----------{K_idx} fold start!----------")
    train_img  , val_img   = Subset(train_imgs, train_idx),  Subset(train_imgs, val_idx)
    train_label, val_label = Subset(train_labels, train_idx),  Subset(train_labels, val_idx)

    dataloaders = {}

    # Train
    train_dataset = Custom_dataset_Albumentations(img_list = train_img,
                                                  labels     = train_label,
                                                  transform  = albumentations_transform,
                                                  mode = 'train')
    dataloaders['train'] = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

    # Val
    val_dataset = Custom_dataset_Albumentations(img_list = val_img,
                                                labels     = val_label,
                                                transform  = albumentations_transform,
                                                mode = 'train')
    dataloaders['vaild'] = DataLoader(val_dataset, shuffle=True, batch_size=batch_size)



    gc.collect()
    torch.cuda.empty_cache()

    best_kfold = 0
    best_f1 = 0

    model = Network().to(device)
    base_optimizer = torch.optim.SGD
    optimizer_ft = SAM(model.parameters(), base_optimizer, lr=1e-6, momentum=0.9) # momentum?
    lr_scheduler = CosineAnnealingWarmUpRestarts(optimizer_ft, T_0=20, T_mult=1, eta_max=0.1,  T_up=4, gamma=0.5)

    # early_stopping = 0
    if (K_idx == start_fold) & (restart_state == 'y') : # 중간에 끊긴 경우
      model, best_idx, best_loss, best_acc, train_loss, train_acc, valid_loss, valid_acc, total_f1  = re_train_model(model, criterion, optimizer_ft, lr_scheduler, num_epochs, K_idx)
    else:
      model, best_idx, best_loss, best_acc, train_loss, train_acc, valid_loss, valid_acc, total_f1  = train_model(model, criterion, optimizer_ft, lr_scheduler, num_epochs, K_idx)


    fold_time_elapsed = time.time() - total_since # 전체누적시간
    #print('one fold - Training complete in {:.0f}m {:.0f}s'.format(fold_time_elapsed // 60, fold_time_elapsed % 60))

    if K_idx+1 != n_splits:
      print('Accumulate training complete in {:.0f}m {:.0f}s'.format(fold_time_elapsed // 60, fold_time_elapsed % 60))
    else:
      print('Total training complete in {:.0f}m {:.0f}s'.format(fold_time_elapsed // 60, fold_time_elapsed % 60))

  else:
    print(f'start_fold : {K_idx} - fold pass')

0
y / n 입력하시오 :n
----------0 fold start!----------
### KFold 0 / 4 ###
Epoch 0/29
----------
train Loss: 4.60840 Acc: 1.03596 macro-f1: 0.00591
vaild Loss: 4.55789 Acc: 0.42857 macro-f1: 0.00530
Accumulate epochs training time : 6m 47s


Epoch 1/29
----------
train Loss: 1.04890 Acc: 74.97023 macro-f1: 0.40665
vaild Loss: 0.53220 Acc: 85.14286 macro-f1: 0.46135
Accumulate epochs training time : 13m 40s


Epoch 2/29
----------
train Loss: 0.57066 Acc: 83.82948 macro-f1: 0.64245
vaild Loss: 0.26304 Acc: 92.00000 macro-f1: 0.67959
Accumulate epochs training time : 20m 32s


Epoch 3/29
----------
train Loss: 0.41956 Acc: 87.33032 macro-f1: 0.73741
vaild Loss: 0.19127 Acc: 95.00000 macro-f1: 0.76755
Accumulate epochs training time : 27m 25s


Epoch 4/29
----------
train Loss: 0.38612 Acc: 88.96166 macro-f1: 0.77278
vaild Loss: 0.26116 Acc: 93.09524 macro-f1: 0.78450
Accumulate epochs training time : 34m 18s


Epoch 5/29
----------
train Loss: 0.31422 Acc: 90.81924 macro-f1: 0.80599
vaild Lo

KeyboardInterrupt: ignored

n## 결과 그래프 그리기

In [ ]:
# ## 결과 그래프 그리기
# print('best model : %d - %1.f / %.1f'%(best_idx, valid_acc[best_idx], valid_loss[best_idx]))
# fig, ax1 = plt.subplots()

# ax1.plot(train_acc, 'b-')
# ax1.plot(valid_acc, 'r-')
# plt.plot(best_idx, valid_acc[best_idx], 'ro')
# ax1.set_xlabel('epoch')
# # Make the y-axis label, ticks and tick labels match the line color.
# ax1.set_ylabel('acc', color='k')
# ax1.tick_params('y', colors='k')

# ax2 = ax1.twinx()
# ax2.plot(train_loss, 'g-')
# ax2.plot(valid_loss, 'k-')
# plt.plot(best_idx, valid_loss[best_idx], 'ro')
# ax2.set_ylabel('loss', color='k')
# ax2.tick_params('y', colors='k')

# fig.tight_layout()
# plt.show()

kfold 결과

In [6]:
# 모델을 평가 모드로 변경하고 test 데이터 분류
test_png = sorted(glob('/content/drive/Othercomputers/내 MacBook Pro/open/test/*.png'))
test_imgs = [img_load(n) for n in tqdm(test_png)]
test_dataset = Custom_dataset_Albumentations(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)
model_version = 'efficientNet_v2_last'
pred_ensemble = [] # 모든 폴드 결과값
#/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/efficientnetb4_ver4/efficientNet_b4_ver4_final_best_model_0fold.pth
device = torch.device('cuda')
import timm
for i in range(4): 
  pred_prob = [] # 각 폴드별 결과값
  model_test = Network(mode = 'test').to(device)
  path = f'/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/{model_version}_final_best_model_{i}fold.pth'
  checkpoint = torch.load(path)
  model_test.load_state_dict(checkpoint['state_dict'])
  model_test.eval()
  with torch.no_grad():
      for batch in (test_loader):
          x = torch.tensor(batch[0], dtype = torch.float32, device = device)
          with torch.cuda.amp.autocast():
              pred = model_test(x)
              pred_prob.extend(pred.detach().cpu().numpy())
      pred_ensemble.append(pred_prob)

print(len(pred_ensemble))

100%|██████████| 2154/2154 [55:58<00:00,  1.56s/it]


4


In [7]:
train_y = pd.read_csv("/content/drive/Othercomputers/내 MacBook Pro/open/train_df.csv")

train_labels = train_y["label"] # 레이블순서는 이미지 파일 순서대로임

## 레이블별 키값
label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))} # 오름차순으로 레이블별로 숫자 부여(0부터 시작)

train_labels = [label_unique[k] for k in tqdm(train_labels)]

pred = (np.array(pred_ensemble[0])+ np.array(pred_ensemble[1])+np.array(pred_ensemble[2])+ np.array(pred_ensemble[3]))/4
f_pred = np.array(pred).argmax(1).tolist()

## 숫자로된 레이블을 문자열 레이블로 변경
label_decoder = {value:key for key, value in label_unique.items()}
f_result = [label_decoder[result] for result in f_pred]

from datetime import datetime as dt 
today = dt.today().strftime('%Y-%m-%d')
version = f'efficientNet_v2_last_{today}'
submission = pd.read_csv(f"/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/sample_submission.csv")

## 시각적 확인을 위해 문자열 레이블로 이루어진 된 label 필드 생성
submission["label"] = f_result
display(submission)
submission.to_csv(f"/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/sample_submission_{version}.csv", index = False)

100%|██████████| 4277/4277 [00:00<00:00, 2437700.53it/s]


,index,label
0,0,tile-glue_strip
1,1,grid-good
2,2,transistor-good
3,3,tile-gray_stroke
4,4,tile-good
...,...,...
2149,2149,tile-gray_stroke
2150,2150,screw-good
2151,2151,grid-good
2152,2152,cable-good


## k-fold 결과 - 모델추론(두모델 합본)

In [ ]:
# torch.save({'epoch':epoch,
#             'state_dict':model.state_dict(),
#             'optimizer': optimizer.state_dict(),
#             'best_model_wts':best_model_wts,
#             'best_idx':best_idx,
#             'best_acc':best_acc,
#             'best_loss':best_loss,
#             'train_loss' : train_loss,
#             'train_acc' : train_acc,
#             'valid_loss' : valid_loss,
#             'valid_acc' : valid_acc,
#             'total_f1' : total_f1,},
#             #'scaler': scaler.state_dict(),
#               f'/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/{model_version}_model_{K_idx}fold.pth') 
#     model.load_state_dict(best_model_wts)

In [ ]:
# torch.save({'state_dict':model.state_dict(),
#                 'train_loss' : train_loss,
#                 'train_acc' : train_acc,
#                 'valid_loss' : valid_loss,
#                 'valid_acc' : valid_acc,
#                 'total_f1' : total_f1,},
#                 f'/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/{model_version}_final_best_model_{K_idx}fold.pth') 

In [ ]:
# ## 기본 모델 및 옵티마이저
# model = Network().to(device)
# base_optimizer = torch.optim.SGD
# optimizer_ft = SAM(model.parameters(), base_optimizer, lr=1e-6, momentum=0.9) # momentum?


# model_version = 'efficientNet_b4_ver3'
# K_idx = '1'

# PATH1 =  f'/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/{model_version}_model_{K_idx}fold.pth'
# PATH2 =  f'/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/{model_version}_final_best_model_{K_idx}fold.pth'

# checkpoint = torch.load(PATH1)
# model.load_state_dict(checkpoint['best_model_wts'])
# model.eval()

In [ ]:
# class Network_all(nn.Module):
#     def __init__(self,mode = 'train', modelType = 'b4'):
#         super(Network_all, self).__init__()
#         self.mode = mode
#         self.modelType = modelType
#         if self.mode == 'train':
#           if self.modelType == 'b4':
#             self.model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=88, drop_path_rate = 0.3)
#           else:
#             self.model = timm.create_model('efficientnetv2_rw_s', pretrained=True, num_classes=88, drop_path_rate = 0.3)

#         if self.mode == 'test':
#           if self.modelType == 'b4':
#             self.model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=88, drop_path_rate = 0)
#           else:
#             self.model = timm.create_model('efficientnetv2_rw_s', pretrained=True, num_classes=88, drop_path_rate =0)
        
#     def forward(self, x):
#         x = self.model(x)
#         return x

# class Custom_dataset_test(Dataset):
#     def __init__(self, img_paths, labels, mode='train'):
#         self.img_paths = img_paths
#         self.labels = labels
#         self.mode=mode
#     def __len__(self):
#         return len(self.img_paths)

#     def __getitem__(self, idx):
#         if self.mode=='test':
#           img = self.img_paths[idx]
#           img = transforms.ToTensor()(img)
#           img = transforms.Normalize(mean=[0.432, 0.402, 0.393], std=[0.182, 0.174 , 0.163])(img) #mean=[0.432, 0.402, 0.393], std=[0.182, 0.174 , 0.163]
#           label = self.labels[idx]
#           return img, label

In [ ]:
## 모델을 평가 모드로 변경하고 test 데이터 분류
test_png = sorted(glob('/content/drive/Othercomputers/내 MacBook Pro/open/test/*.png'))
test_imgs = [img_load(n) for n in tqdm(test_png)]
test_dataset = Custom_dataset_test(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)
#model_version = 'efficientNet_b4_ver5'
pred_ensemble = [] # 모든 폴드 결과값
#/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/efficientnetb4_ver4/efficientNet_b4_ver4_final_best_model_0fold.pth
device = torch.device('cuda')
import timm
for m in ['b4', 'v2'] :
  for i in range(2): 
    pred_prob = [] # 각 폴드별 결과값
    model_test = Network_all(mode = 'test', modelType = m).to(device)
    if m == 'b4':
      dir = 'efficientnetb4_ver6'
      model_version = 'efficientNet_b4_ver6'
      path = f'/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/{dir}/{model_version}_final_best_model_{i}fold.pth'
    else:
      dir = 'efficientnetv2_ver6'
      #efficientnetv2_rw_s_size_up_normal_no_error_data_final_best_model_1fold
      model_version = 'efficientnetv2_rw_s_size_up_normal_no_error_data'
      path = f'/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/{dir}/{model_version}_final_best_model_{i}fold.pth'
    checkpoint = torch.load(path)
    model_test.load_state_dict(checkpoint['state_dict'])
    model_test.eval()
    with torch.no_grad():
        for batch in (test_loader):
            x = torch.tensor(batch[0], dtype = torch.float32, device = device)
            with torch.cuda.amp.autocast():
                pred = model_test(x)
                pred_prob.extend(pred.detach().cpu().numpy())
        pred_ensemble.append(pred_prob)

print(len(pred_ensemble))

100%|██████████| 2154/2154 [01:07<00:00, 31.73it/s]


4


In [ ]:
train_y = pd.read_csv("/content/drive/Othercomputers/내 MacBook Pro/open/train_df.csv")

train_labels = train_y["label"] # 레이블순서는 이미지 파일 순서대로임

## 레이블별 키값
label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))} # 오름차순으로 레이블별로 숫자 부여(0부터 시작)

train_labels = [label_unique[k] for k in tqdm(train_labels)]

pred = (np.array(pred_ensemble[0])+ np.array(pred_ensemble[1])+np.array(pred_ensemble[2])+ np.array(pred_ensemble[3]))/4
f_pred = np.array(pred).argmax(1).tolist()

## 숫자로된 레이블을 문자열 레이블로 변경
label_decoder = {value:key for key, value in label_unique.items()}
f_result = [label_decoder[result] for result in f_pred]

from datetime import datetime as dt 
today = dt.today().strftime('%Y-%m-%d')
version = f'efficientNet_b4&v2_ver6_2_{today}'
submission = pd.read_csv(f"/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/sample_submission.csv")

## 시각적 확인을 위해 문자열 레이블로 이루어진 된 label 필드 생성
submission["label"] = f_result
display(submission)
submission.to_csv(f"/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/sample_submission_{version}.csv", index = False)

100%|██████████| 4277/4277 [00:00<00:00, 2217433.65it/s]


,index,label
0,0,tile-glue_strip
1,1,grid-good
2,2,transistor-good
3,3,tile-gray_stroke
4,4,tile-good
...,...,...
2149,2149,tile-gray_stroke
2150,2150,screw-good
2151,2151,grid-good
2152,2152,cable-good


In [ ]:
import pandas as pd
submission2 = pd.read_csv("/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/sample_submission_efficientNet_b4&v2_ver6_2022-05-12.csv")
submission_all = pd.merge(submission, submission2, how = 'left', on = 'index')
submission_all


,index,label_x,label_y
0,0,tile-glue_strip,tile-glue_strip
1,1,grid-good,grid-good
2,2,transistor-good,transistor-good
3,3,tile-gray_stroke,tile-gray_stroke
4,4,tile-good,tile-good
...,...,...,...
2149,2149,tile-gray_stroke,tile-gray_stroke
2150,2150,screw-good,screw-good
2151,2151,grid-good,grid-good
2152,2152,cable-good,cable-good


In [ ]:
submission_all[submission_all.label_x != submission_all.label_y]

,index,label_x,label_y
22,22,bottle-broken_large,bottle-broken_small
42,42,screw-thread_top,screw-good
57,57,grid-good,carpet-good
64,64,capsule-scratch,capsule-faulty_imprint
69,69,metal_nut-good,metal_nut-color
...,...,...,...
2045,2045,screw-manipulated_front,screw-good
2078,2078,pill-combined,pill-faulty_imprint
2083,2083,pill-good,pill-faulty_imprint
2102,2102,screw-thread_top,screw-good


In [ ]:
(149 /2154 )*100

6.917363045496751

In [ ]:
## 숫자로된 레이블을 문자열 레이블로 변경
label_decoder = {value:key for key, value in label_unique.items()}
f_result = [label_decoder[result] for result in f_pred]

In [ ]:
from datetime import datetime as dt 
today = dt.today().strftime('%Y-%m-%d')
version = f'efficientNet_b4_sam_edit2_data_ver2_{today}'
submission = pd.read_csv(f"/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/sample_submission.csv")

## 시각적 확인을 위해 문자열 레이블로 이루어진 된 label 필드 생성
submission["label"] = f_result
display(submission)
submission.to_csv(f"/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/sample_submission_{version}.csv", index = False)

In [ ]:
# # 모델 저장
# model_version = f'efficientNet_b4_sam_edit2_data_ver2_{today}'
# torch.save(model.state_dict(), f'/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/{model_version}.pt')

In [ ]:
a = pd.read_csv(f"/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/sample_submission.csv")
a

,index,label
0,0,tile-good
1,1,tile-good
2,2,tile-good
3,3,tile-good
4,4,tile-good
...,...,...
2149,2149,tile-good
2150,2150,tile-good
2151,2151,tile-good
2152,2152,tile-good


In [ ]:
train_y[train_y.state != 'good']

,index,file_name,class,state,label
8,8,10008.png,cable,bent_wire,cable-bent_wire
10,10,10010.png,carpet,hole,carpet-hole
13,13,10013.png,pill,pill_type,pill-pill_type
16,16,10016.png,metal_nut,scratch,metal_nut-scratch
19,19,10019.png,screw,thread_side,screw-thread_side
...,...,...,...,...,...
4245,4245,14245.png,leather,cut,leather-cut
4248,4248,14248.png,transistor,bent_lead,transistor-bent_lead
4251,4251,14251.png,cable,combined,cable-combined
4252,4252,14252.png,pill,color,pill-color


In [ ]:
4277/ (4277 + 4277 + 648 + 648+ 648 + )

0.40741093541626977

In [ ]:
(648*12) / (4277 + 4277 + (648 *10))

0.5172276174005588

In [ ]:
4277 + 4277 + (648 *10)

15034